In [1]:
import sys
from greedy_gd import *
sys.path.append('../../src/')
from utils.datasets import *
from models.gcn import *
from models.trainable import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
cora_dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = cora_dataset[0].to(device)

In [4]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [5]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [6]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 500)

Epoch 0, Train Loss - 4.637295722961426, Val Loss - 2.314457654953003, Val Accuracy - 0.246
Epoch 20, Train Loss - 0.6201252937316895, Val Loss - 1.2113299369812012, Val Accuracy - 0.643
Epoch 40, Train Loss - 0.2878643274307251, Val Loss - 1.19075608253479, Val Accuracy - 0.755
Epoch 60, Train Loss - 0.12992607057094574, Val Loss - 1.7180954217910767, Val Accuracy - 0.746
Epoch 80, Train Loss - 0.15639245510101318, Val Loss - 1.5200910568237305, Val Accuracy - 0.761
Epoch 100, Train Loss - 0.06581344455480576, Val Loss - 1.92869234085083, Val Accuracy - 0.743
Epoch 120, Train Loss - 0.10606756806373596, Val Loss - 1.9283720254898071, Val Accuracy - 0.742
Epoch 140, Train Loss - 0.09140774607658386, Val Loss - 1.8906110525131226, Val Accuracy - 0.738
Epoch 160, Train Loss - 0.036504678428173065, Val Loss - 2.0055768489837646, Val Accuracy - 0.759
Epoch 180, Train Loss - 0.08692602068185806, Val Loss - 1.8264003992080688, Val Accuracy - 0.774
Epoch 200, Train Loss - 0.05655069649219513,

In [7]:
data.edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]], device='cuda:0')

In [8]:
attacker = Metattack(data, device=device)

In [9]:
attacker.setup_surrogate(model,
                         labeled_nodes=data.train_mask,
                         unlabeled_nodes=data.test_mask, lambda_=0.)

In [10]:
attacker.reset()
attacker.attack(0.1)

Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

tensor(0.0971, device='cuda:0') tensor(156766, device='cuda:0')
max weight: 0.0
tensor(0.0801, device='cuda:0') tensor(64539, device='cuda:0')
max weight: 0.0
tensor(0.0730, device='cuda:0') tensor(20547, device='cuda:0')
max weight: 0.0
tensor(0.0719, device='cuda:0') tensor(135109, device='cuda:0')
max weight: 0.0
tensor(0.0784, device='cuda:0') tensor(11332, device='cuda:0')
max weight: 0.0
tensor(0.0778, device='cuda:0') tensor(313932, device='cuda:0')
max weight: 0.0
tensor(0.0728, device='cuda:0') tensor(64981, device='cuda:0')
max weight: 0.0
tensor(0.0771, device='cuda:0') tensor(36163, device='cuda:0')
max weight: 0.0
tensor(0.0777, device='cuda:0') tensor(75955, device='cuda:0')
max weight: 0.0
tensor(0.0846, device='cuda:0') tensor(64797, device='cuda:0')
max weight: 0.0
tensor(0.0757, device='cuda:0') tensor(357350, device='cuda:0')
max weight: 0.0
tensor(0.0633, device='cuda:0') tensor(59279, device='cuda:0')
max weight: 0.0
tensor(0.0636, device='cuda:0') tensor(370891, d

KeyboardInterrupt: 

In [11]:
new_data = handle_new_edges(data, attacker, device)

torch.Size([2, 11592])


In [12]:
loss, acc = train.test(new_data)
print(f'Loss: {loss}, Acc: {acc}')

Loss: 3.3026821613311768, Acc: 0.71


In [15]:
trainer_poison = Trainable(GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device))
trainer_poison.fit(new_data, 500)

Epoch 0, Train Loss - 5.352356433868408, Val Loss - 5.4662699699401855, Val Accuracy - 0.176
Epoch 20, Train Loss - 1.2599878311157227, Val Loss - 2.112591505050659, Val Accuracy - 0.483
Epoch 40, Train Loss - 0.8465968370437622, Val Loss - 2.542515516281128, Val Accuracy - 0.559
Epoch 60, Train Loss - 0.5768699645996094, Val Loss - 3.0360071659088135, Val Accuracy - 0.563
Epoch 80, Train Loss - 0.29088255763053894, Val Loss - 2.876858711242676, Val Accuracy - 0.589
Epoch 100, Train Loss - 0.19641634821891785, Val Loss - 2.9545750617980957, Val Accuracy - 0.58
Epoch 120, Train Loss - 0.16808176040649414, Val Loss - 3.40596866607666, Val Accuracy - 0.59
Epoch 140, Train Loss - 0.08979274332523346, Val Loss - 3.5005459785461426, Val Accuracy - 0.574
Epoch 160, Train Loss - 0.10731571167707443, Val Loss - 3.7003884315490723, Val Accuracy - 0.593
Epoch 180, Train Loss - 0.10311038047075272, Val Loss - 3.7477071285247803, Val Accuracy - 0.593
Epoch 200, Train Loss - 0.07701707631349564, Val

In [14]:
loss, acc = trainer_poison.test(new_data)
print(f'Loss: {loss}, Acc: {acc}')

Loss: 3.2340550422668457, Acc: 0.594
